# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770293050080                   -0.53    9.0         
  2   -2.771681123412       -2.86       -1.30    1.0    231ms
  3   -2.771713758763       -4.49       -2.71    1.0    164ms
  4   -2.771714703538       -6.02       -3.53    2.0    198ms
  5   -2.771714712654       -8.04       -3.84    2.0    231ms
  6   -2.771714715209       -8.59       -5.58    1.0    173ms
  7   -2.771714715248      -10.41       -5.50    3.0    235ms
  8   -2.771714715250      -11.94       -6.06    1.0    176ms
  9   -2.771714715250      -13.37       -6.49    1.0    204ms
 10   -2.771714715250      -13.92       -7.02    1.0    180ms
 11   -2.771714715250   +  -13.95       -7.71    2.0    209ms
 12   -2.771714715250      -13.91       -7.91    2.0    206ms
 13   -2.771714715250      -15.05       -8.70    1.0    194ms


-0.00013457360763584832

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770494286651                   -0.52    8.0         
  2   -2.771777339354       -2.89       -1.32    1.0    166ms
  3   -2.771801428963       -4.62       -2.47    1.0    183ms
  4   -2.771801973203       -6.26       -3.18    1.0    196ms
  5   -2.771802074099       -7.00       -4.10    2.0    187ms
  6   -2.771802074064   +  -10.47       -4.12    1.0    204ms
  7   -2.771802074436       -9.43       -4.73    1.0    170ms
  8   -2.771802074455      -10.74       -4.81    1.0    212ms
  9   -2.771802074474      -10.71       -5.51    1.0    188ms
 10   -2.771802074476      -11.77       -5.95    2.0    201ms
 11   -2.771802074476      -12.84       -6.23    1.0    197ms
 12   -2.771802074476      -13.53       -6.68    1.0    196ms
 13   -2.771802074476   +  -14.10       -7.53    2.0    233ms
 14   -2.771802074476   +  -14.65       -8.10    1.0    202ms


0.01761222055253888

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457360763584832
Displaced dipole:  0.01761222055253888
Polarizability :   1.7746794160174728


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920874994e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551665536e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852768417577e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602666467e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088788050328e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.287951121232e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.123603288686e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.123603288686e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 6.847865755472e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.282230649684e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.435962134818e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.435962134818e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.4359559154035014e-13
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.